In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
import pandas as pd
import numpy as np

# 1. Load the dataset
file_path = "/content/drive/MyDrive/Credit Card Data.csv"   # update path as needed
df = pd.read_csv(file_path)

# 2. Basic cleaning
# Ensure correct dtypes
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])
df['dob'] = pd.to_datetime(df['dob'])

# Create some helper features
df['age'] = (df['trans_date_trans_time'].dt.year - df['dob'].dt.year)
df['tx_hour'] = df['trans_date_trans_time'].dt.hour
df['tx_dayofweek'] = df['trans_date_trans_time'].dt.dayofweek  # 0=Mon

# Sort by user + time
df = df.sort_values(['cc_num', 'trans_date_trans_time']).reset_index(drop=True)


In [39]:
# Filter to legitimate transactions
df_legit = df[df['is_fraud'] == 0].copy()

def build_user_profiles(df_legit):
    # Basic numeric stats
    agg_numeric = df_legit.groupby('cc_num').agg(
        n_legit_tx=('amt', 'count'),
        total_spent=('amt', 'sum'),
        mean_spent=('amt', 'mean'),
        std_spent=('amt', 'std'),
        min_spent=('amt', 'min'),
        max_spent=('amt', 'max'),
        median_spent=('amt', 'median'),
        age_mean=('age', 'mean'),
        city_pop_mean=('city_pop', 'mean')
    )

    # Category distribution (% of transactions by category)
    cat_counts = (
        df_legit.groupby(['cc_num', 'category'])['amt']
        .count()
        .unstack(fill_value=0)
    )
    cat_pct = cat_counts.div(cat_counts.sum(axis=1), axis=0)
    cat_pct.columns = [f"cat_pct_{c}" for c in cat_pct.columns]

    # Time-of-day distribution
    # Buckets: night(0-6), morning(6-12), afternoon(12-18), evening(18-24)
    def time_bucket(h):
        if 0 <= h < 6:
            return 'night'
        elif 6 <= h < 12:
            return 'morning'
        elif 12 <= h < 18:
            return 'afternoon'
        else:
            return 'evening'

    df_legit['time_bucket'] = df_legit['tx_hour'].apply(time_bucket)

    tb_counts = (
        df_legit.groupby(['cc_num', 'time_bucket'])['amt']
        .count()
        .unstack(fill_value=0)
    )
    tb_pct = tb_counts.div(tb_counts.sum(axis=1), axis=0)
    tb_pct.columns = [f"tb_pct_{c}" for c in tb_pct.columns]

    # Geography: main state/city, avg lat/long
    geo_agg = df_legit.groupby('cc_num').agg(
        main_state=('state', lambda x: x.value_counts().idxmax()),
        main_city=('city', lambda x: x.value_counts().idxmax()),
        mean_lat=('lat', 'mean'),
        mean_long=('long', 'mean')
    )

    # Merge all
    user_profiles = (
        agg_numeric
        .join(cat_pct, how='left')
        .join(tb_pct, how='left')
        .join(geo_agg, how='left')
    ).reset_index()  # keep cc_num as a column

    return user_profiles

user_profiles = build_user_profiles(df_legit)
user_profiles.head()


,cc_num,n_legit_tx,total_spent,mean_spent,std_spent,min_spent,max_spent,median_spent,age_mean,city_pop_mean,...,cat_pct_shopping_pos,cat_pct_travel,tb_pct_afternoon,tb_pct_evening,tb_pct_morning,tb_pct_night,main_state,main_city,mean_lat,mean_long
0,60416207185,1509,83143.74,55.098569,120.422174,1.02,3075.09,36.66,33.277005,1645.0,...,0.098741,0.019881,0.361829,0.332671,0.157058,0.148443,WY,Fort Washakie,43.0048,-108.8964
1,60422928733,1519,98140.02,64.608308,83.453978,1.04,1290.37,52.15,77.276498,46944.0,...,0.099408,0.036208,0.205398,0.196182,0.292298,0.306122,SC,North Augusta,33.6028,-81.9748
2,60423098130,500,54075.55,108.151100,1211.999695,1.01,27119.77,34.68,61.266000,83.0,...,0.052000,0.044000,0.258000,0.292000,0.236000,0.214000,OK,Amorita,36.9412,-98.2458
3,60427851591,514,49483.82,96.272023,90.327352,20.74,569.40,76.36,43.278210,2142.0,...,0.091440,0.040856,0.268482,0.233463,0.284047,0.214008,OK,Burns Flat,35.3492,-99.1880
4,60487002085,496,25160.11,50.726028,65.843969,1.02,750.39,35.64,47.360887,233060.0,...,0.070565,0.028226,0.332661,0.344758,0.167339,0.155242,MS,Jackson,32.3739,-90.1293


In [40]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report

# Merge user profiles back to all transactions
df_model = df.merge(
    user_profiles,
    on='cc_num',
    how='left',
    suffixes=('', '_prof')
)

# Select columns for the classifier
feature_cols_cat = ['category', 'state', 'city', 'job', 'gender']
feature_cols_num = [
    'amt', 'age', 'city_pop', 'tx_hour', 'tx_dayofweek',
    'n_legit_tx', 'total_spent', 'mean_spent', 'std_spent',
    'min_spent', 'max_spent', 'median_spent',
    'mean_lat', 'mean_long'
] + [c for c in df_model.columns if c.startswith('cat_pct_') or c.startswith('tb_pct_')]

# Label encode categorical vars
df_model_enc = df_model.copy()
label_encoders = {}

for col in feature_cols_cat:
    le = LabelEncoder()
    # Get all unique values from original data
    original_classes = df_model_enc[col].astype(str).unique()
    # Add a placeholder for unknown values that might appear in synthetic data
    # Ensuring 'UNKNOWN_CATEGORY' is part of the classes the encoder knows
    combined_classes = np.append(original_classes, 'UNKNOWN_CATEGORY')
    le.fit(combined_classes) # Fit on all possible original + 'UNKNOWN_CATEGORY'

    df_model_enc[col] = le.transform(df_model_enc[col].astype(str))
    label_encoders[col] = le

X = df_model_enc[feature_cols_num + feature_cols_cat]
y = df_model_enc['is_fraud'].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [41]:
from sklearn.ensemble import RandomForestClassifier

clf_baseline = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    n_jobs=-1,
    random_state=42,
    class_weight='balanced'  # handle class imbalance
)
clf_baseline.fit(X_train, y_train)

y_pred = clf_baseline.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

fpr_baseline = fp / (fp + tn)
print("Confusion matrix:\n", cm)
print("Baseline FPR:", fpr_baseline)
print(classification_report(y_test, y_pred, digits=4))

Confusion matrix:
 [[257806     28]
 [   396   1105]]
Baseline FPR: 0.00010859700427406781
              precision    recall  f1-score   support

           0     0.9985    0.9999    0.9992    257834
           1     0.9753    0.7362    0.8390      1501

    accuracy                         0.9984    259335
   macro avg     0.9869    0.8680    0.9191    259335
weighted avg     0.9983    0.9984    0.9983    259335



In [42]:
def profile_to_text(row):
    # Use safe fills for NaNs
    main_state = row.get('main_state', 'Unknown')
    main_city = row.get('main_city', 'Unknown')
    mean_spent = row.get('mean_spent', 0)
    max_spent = row.get('max_spent', 0)
    n_tx = row.get('n_legit_tx', 0)

    # a few top category prefs
    cat_cols = [c for c in user_profiles.columns if c.startswith('cat_pct_')]
    top_cats = sorted(
        [(c.replace('cat_pct_', ''), row[c]) for c in cat_cols],
        key=lambda x: x[1],
        reverse=True
    )[:3]
    cat_str = ", ".join([f"{name} ({pct:.0%})" for name, pct in top_cats if pct > 0])

    profile = (
        f"User {int(row['cc_num'])} lives mainly in {main_city}, {main_state}. "
        f"They have {int(n_tx)} legitimate transactions. "
        f"The average transaction amount is ${mean_spent:.2f} and max is ${max_spent:.2f}. "
    )
    if cat_str:
        profile += f"The main spending categories are: {cat_str}. "
    return profile

user_profiles['profile_text'] = user_profiles.apply(profile_to_text, axis=1)


In [43]:
df_fraud = df[df['is_fraud'] == 1].copy()

# Join profiles to fraud rows
df_fraud = df_fraud.merge(
    user_profiles[['cc_num', 'profile_text']],
    on='cc_num',
    how='left'
)

def tx_to_text(row):
    # Convert a single transaction row into a structured text / pseudo-JSON
    return (
        "{"
        f" 'trans_date_trans_time': '{row['trans_date_trans_time']}',"
        f" 'cc_num': '{int(row['cc_num'])}',"
        f" 'merchant': '{row['merchant']}',"
        f" 'category': '{row['category']}',"
        f" 'amt': {row['amt']:.2f},"
        f" 'city': '{row['city']}',"
        f" 'state': '{row['state']}',"
        f" 'zip': '{row['zip']}',"
        f" 'lat': {row['lat']},"
        f" 'long': {row['long']},"
        f" 'city_pop': {row['city_pop']},"
        f" 'job': '{row['job']}',"
        f" 'dob': '{row['dob']}',"
        f" 'unix_time': {row['unix_time']},"
        f" 'merch_lat': {row['merch_lat']},"
        f" 'merch_long': {row['merch_long']},"
        f" 'merch_zipcode': '{row['merch_zipcode']}',"
        f" 'is_fraud': 1"
        " }"
    )

prompts = []
completions = []

for _, row in df_fraud.iterrows():
    profile_text = row['profile_text']
    prompt = (
        "You are a fraud data generator. Given the following user profile, "
        "generate ONE realistic fraudulent credit card transaction for this user. "
        "Return ONLY a Python-style dictionary with keys matching the dataset, "
        "and set 'is_fraud' to 1.\n\n"
        f"USER PROFILE:\n{profile_text}\n\n"
        "FRAUD TRANSACTION:\n"
    )
    completion = tx_to_text(row)
    prompts.append(prompt)
    completions.append(completion)

train_df_llm = pd.DataFrame({
    "prompt": prompts,
    "completion": completions
})
train_df_llm.head()


,prompt,completion
0,You are a fraud data generator. Given the foll...,{ 'trans_date_trans_time': '2019-03-01 01:32:5...
1,You are a fraud data generator. Given the foll...,{ 'trans_date_trans_time': '2019-03-01 02:42:2...
2,You are a fraud data generator. Given the foll...,{ 'trans_date_trans_time': '2019-03-01 23:06:5...
3,You are a fraud data generator. Given the foll...,{ 'trans_date_trans_time': '2019-03-02 22:10:3...
4,You are a fraud data generator. Given the foll...,{ 'trans_date_trans_time': '2019-03-02 22:10:5...


In [44]:
!pip install -q transformers datasets peft accelerate bitsandbytes


In [45]:
from datasets import Dataset

def format_example(ex):
    # Simple instruction-tuning style
    text = ex["prompt"] + ex["completion"]
    return {"text": text}

hf_ds = Dataset.from_pandas(train_df_llm[['prompt', 'completion']])
hf_ds = hf_ds.map(format_example, remove_columns=['prompt', 'completion'])


Map:   0%|          | 0/7506 [00:00<?, ? examples/s]

In [23]:
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
import torch

model_name = "microsoft/phi-2"  # or another small causal LM

tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize_fn(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )

tokenized_ds = hf_ds.map(tokenize_fn, batched=True, remove_columns=['text'])

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Use BitsAndBytesConfig as recommended by the warning and for better device handling
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16, # or torch.bfloat16 if GPU supports it
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": 0} # Explicitly set device to GPU 0
)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],  # adjust depending on model
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

training_args = TrainingArguments(
    output_dir="./fraud-llm",
    per_device_train_batch_size=2, # Reduced batch size
    gradient_accumulation_steps=4,
    num_train_epochs=0.5,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=50,
    save_total_limit=2,
    save_steps=500,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds,
    data_collator=data_collator
)

trainer.train()

# Save adapter
trainer.save_model("./fraud-llm")
tokenizer.save_pretrained("./fraud-llm")

Map:   0%|          | 0/7506 [00:00<?, ? examples/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 2,621,440 || all params: 2,782,305,280 || trainable%: 0.0942


/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
50,1.657400
100,0.905900
150,0.860800
200,0.842400
250,0.833200
300,0.825100
350,0.821000
400,0.814900
450,0.810800


('./fraud-llm/tokenizer_config.json',
 './fraud-llm/special_tokens_map.json',
 './fraud-llm/vocab.json',
 './fraud-llm/merges.txt',
 './fraud-llm/added_tokens.json',
 './fraud-llm/tokenizer.json')

In [46]:
import torch
import ast  # to safely parse the Python dict string

# Reload model + tokenizer (with LoRA)
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto"
)
model = PeftModel.from_pretrained(base_model, "./fraud-llm")
tokenizer = AutoTokenizer.from_pretrained("./fraud-llm")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def generate_fraud_for_user(profile_text, num_samples=10, max_new_tokens=256):
    prompt = (
        "You are a fraud data generator. Given the following user profile, "
        "generate ONE realistic fraudulent credit card transaction for this user. "
        "Return ONLY a Python-style dictionary with keys matching the dataset, "
        "and set 'is_fraud' to 1.\n\n"
        f"USER PROFILE:\n{profile_text}\n\n"
        "FRAUD TRANSACTION:\n"
    )
    input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **input_ids,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.9,
        top_p=0.95,
        num_return_sequences=num_samples
    )

    gen_texts = []
    for i in range(num_samples):
        text = tokenizer.decode(outputs[i], skip_special_tokens=True)
        # Extract only the dictionary part (after the prompt)
        dict_str = text.split("FRAUD TRANSACTION:\n")[-1].strip()
        gen_texts.append(dict_str)
    return gen_texts


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [47]:
import torch
import ast  # to safely parse the Python dict string

# Reload model + tokenizer (with LoRA)
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto"
)
model = PeftModel.from_pretrained(base_model, "./fraud-llm")
tokenizer = AutoTokenizer.from_pretrained("./fraud-llm")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def generate_fraud_for_user(profile_text, num_samples=3, max_new_tokens=256):
    prompt = (
        "You are a fraud data generator. Given the following user profile, "
        "generate ONE realistic fraudulent credit card transaction for this user. "
        "Return ONLY a Python-style dictionary with keys matching the dataset, "
        "and set 'is_fraud' to 1.\n\n"
        f"USER PROFILE:\n{profile_text}\n\n"
        "FRAUD TRANSACTION:\n"
    )
    input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **input_ids,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.9,
        top_p=0.95,
        num_return_sequences=num_samples
    )

    gen_texts = []
    for i in range(num_samples):
        text = tokenizer.decode(outputs[i], skip_special_tokens=True)
        # Extract only the dictionary part (after the prompt)
        dict_str = text.split("FRAUD TRANSACTION:\n")[-1].strip()
        gen_texts.append(dict_str)
    return gen_texts


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [48]:
synthetic_rows = []

# Example: generate for 5% of users to start
subset_profiles = user_profiles.sample(frac=0.1, random_state=42)

for _, row in subset_profiles.iterrows():
    profile_text = row['profile_text']
    cc_num = row['cc_num']
    gen_list = generate_fraud_for_user(profile_text, num_samples=2)

    for dict_str in gen_list:
        try:
            tx_dict = ast.literal_eval(dict_str)
            tx_dict['cc_num'] = cc_num  # ensure correct user id
            # enforce fraud label
            tx_dict['is_fraud'] = 1
            synthetic_rows.append(tx_dict)
        except Exception as e:
            print("Parse error, skipping:", e)

df_synth = pd.DataFrame(synthetic_rows)
df_synth.head()


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Parse error, skipping: unterminated string literal (detected at line 4) (<unknown>, line 4)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Parse error, skipping: invalid syntax (<unknown>, line 3)
Parse error, skipping: invalid syntax (<unknown>, line 5)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Parse error, skipping: unterminated triple-quoted string literal (detected at line 10) (<unknown>, line 3)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Parse error, skipping: invalid syntax (<unknown>, line 4)
Parse error, skipping: unterminated string literal (detected at line 3) (<unknown>, line 3)
Parse error, skipping: invalid syntax (<unknown>, line 3)


,trans_date_trans_time,cc_num,merchant,category,amt,city,state,zip,lat,long,city_pop,job,dob,unix_time,merch_lat,merch_long,merch_zipcode,is_fraud
0,2019-03-08 22:38:46,4477156602511939689,"fraud_Buckholt, Witter and Burch",grocery_pos,292.57,American Fork,UT,84007,41.3601,-112.1429,349437,"Engineer, water/wastewater",1977-10-15 00:00:00,1332269886,42.662093,-113.894039,84102.0,1
1,2019-10-02 22:06:49,375248307838179,fraud_Dwyer-Henderson,shopping_net,879.43,Tallmansville,WV,26080,39.2462,-81.5501,1398,"Nurse, community",1946-01-12 00:00:00,1345782919,40.588067,-81.296979,25918.0,1
2,2019-03-04 22:08:10,375248307838179,fraud_Suttor-Grimm,misc_net,771.21,Tallmansville,WV,25112,39.3344,-80.0526,1541,Engineering geologist,1932-06-14 00:00:00,1356759810,39.896682,-80.867894,25142.0,1
3,2019-01-03 01:03:27,6011893664860915,fraud_Mills-Welch,shopping_net,1188.77,Littleton,CO,80001,40.3863,-106.4971,292539,"Librarian, public",1987-03-31 00:00:00,1358567517,41.788889,-105.068981,84849.0,1
4,2019-03-31 01:20:13,4208110975550360171,fraud_Vogt-Pauly,entertainment,5.49,West Green,GA,31757,34.2434,-83.1579,1798,Physician,1931-03-30 00:00:00,1350395733,33.936167,-83.332479,32516.0,1


In [49]:
# Ensure all expected columns exist
expected_cols = df.columns  # from original dataset

for col in expected_cols:
    if col not in df_synth.columns:
        df_synth[col] = np.nan

df_synth = df_synth[expected_cols]


In [50]:
# Combine original training set rows (using indices from earlier split) with synthetic rows
# First, we reconstruct X_train / y_train with indices

train_indices = X_train.index
df_train_original = df_model_enc.loc[train_indices].copy()

# Re-encode synthetic data with the same label encoders
df_synth_enc = df_synth.copy()

# Convert dates and create helper features if needed, matching preprocessing
df_synth_enc['trans_date_trans_time'] = pd.to_datetime(df_synth_enc['trans_date_trans_time'])
df_synth_enc['dob'] = pd.to_datetime(df_synth_enc['dob'])
df_synth_enc['age'] = df_synth_enc['trans_date_trans_time'].dt.year - df_synth_enc['dob'].dt.year
df_synth_enc['tx_hour'] = df_synth_enc['trans_date_trans_time'].dt.hour
df_synth_enc['tx_dayofweek'] = df_synth_enc['trans_date_trans_time'].dt.dayofweek

# Merge profile features for synthetic rows too
df_synth_enc = df_synth_enc.merge(
    user_profiles,
    on='cc_num',
    how='left',
    suffixes=('', '_prof')
)

# Encode categorical features with existing label encoders
for col, le in label_encoders.items():
    # Ensure the column is of string type before processing to prevent FutureWarning
    df_synth_enc[col] = df_synth_enc[col].astype(str)

    # Replace unseen labels in df_synth_enc with 'UNKNOWN_CATEGORY' before transforming
    # Use .isin() for boolean indexing for efficiency and to handle potential NaNs safely
    unseen_mask = ~df_synth_enc[col].isin(le.classes_)
    if unseen_mask.any(): # Check if there are any unseen labels
        df_synth_enc.loc[unseen_mask, col] = 'UNKNOWN_CATEGORY'
    df_synth_enc[col] = le.transform(df_synth_enc[col])

# Build X_synth, y_synth
X_synth = df_synth_enc[feature_cols_num + feature_cols_cat]
y_synth = df_synth_enc['is_fraud'].astype(int)

# Concatenate training data
X_train_aug = pd.concat([X_train, X_synth], axis=0)
y_train_aug = pd.concat([y_train, y_synth], axis=0)

In [51]:
clf_aug = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    n_jobs=-1,
    random_state=42,
    class_weight='balanced'
)
clf_aug.fit(X_train_aug, y_train_aug)

y_pred_aug = clf_aug.predict(X_test)

cm_aug = confusion_matrix(y_test, y_pred_aug)
tn_a, fp_a, fn_a, tp_a = cm_aug.ravel()
fpr_aug = fp_a / (fp_a + tn_a)

print("Confusion matrix (augmented):\n", cm_aug)
print("Augmented FPR:", fpr_aug)
print("Change in FPR:", fpr_aug - fpr_baseline)
print(classification_report(y_test, y_pred_aug, digits=4))


Confusion matrix (augmented):
 [[257803     31]
 [   390   1111]]
Augmented FPR: 0.0001202323975891465
Change in FPR: 1.1635393315078687e-05
              precision    recall  f1-score   support

           0     0.9985    0.9999    0.9992    257834
           1     0.9729    0.7402    0.8407      1501

    accuracy                         0.9984    259335
   macro avg     0.9857    0.8700    0.9199    259335
weighted avg     0.9983    0.9984    0.9983    259335



In [52]:
schema_description = """
You are a data analyst working with a pandas DataFrame named df.
df has one row per credit card transaction, with these columns:

- row_index: integer row id
- trans_date_trans_time: datetime of transaction
- cc_num: credit card number (user identifier)
- merchant: merchant name
- category: merchant category (e.g., 'grocery', 'gas_transport')
- amt: transaction amount (float)
- first, last: cardholder first and last name
- gender: 'M' or 'F'
- street, city, state, zip: cardholder address
- lat, long: cardholder location coordinates
- city_pop: population of cardholder's city
- job: cardholder job title
- dob: date of birth
- trans_num: unique transaction ID
- unix_time: unix timestamp of transaction
- merch_lat, merch_long: merchant coordinates
- merch_zipcode: merchant zipcode
- is_fraud: 1 if transaction is fraudulent, 0 otherwise

There is also a DataFrame user_profiles with one row per cc_num (user) and columns like:
- cc_num
- n_legit_tx, total_spent, mean_spent, max_spent, min_spent, median_spent
- mean_lat, mean_long
- main_city, main_state
- cat_pct_<category> (fraction of legit txns in that category)
- tb_pct_morning, tb_pct_afternoon, tb_pct_evening, tb_pct_night (time-of-day fractions)
"""
print(schema_description)



You are a data analyst working with a pandas DataFrame named df.
df has one row per credit card transaction, with these columns:

- row_index: integer row id
- trans_date_trans_time: datetime of transaction
- cc_num: credit card number (user identifier)
- merchant: merchant name
- category: merchant category (e.g., 'grocery', 'gas_transport')
- amt: transaction amount (float)
- first, last: cardholder first and last name
- gender: 'M' or 'F'
- street, city, state, zip: cardholder address
- lat, long: cardholder location coordinates
- city_pop: population of cardholder's city
- job: cardholder job title
- dob: date of birth
- trans_num: unique transaction ID
- unix_time: unix timestamp of transaction
- merch_lat, merch_long: merchant coordinates
- merch_zipcode: merchant zipcode
- is_fraud: 1 if transaction is fraudulent, 0 otherwise

There is also a DataFrame user_profiles with one row per cc_num (user) and columns like:
- cc_num
- n_legit_tx, total_spent, mean_spent, max_spent, min_s

In [56]:
# We'll reuse gen_model and tokenizer from previous steps.
# If you are in a fresh runtime, reload them as in Steps 9–11 first.

def generate_pandas_code(question, max_new_tokens=256):
    system_prompt = f"""
{schema_description}

You write ONLY executable Python code, without any explanation, comments, or backticks.

Rules:
- Use the pandas DataFrame df (raw data) and user_profiles when needed.
- Do NOT import any modules (no import statements).
- The final line of your code MUST assign the answer to a variable named result.
- result can be a scalar, Series, DataFrame, or dictionary.
- Use idiomatic pandas, e.g., groupby, sort_values, head, etc.
- NEVER print anything.
- Do NOT call input(), open(), eval(), exec(), or OS functions.
"""
    prompt = system_prompt + "\n\nQuestion:\n" + question + "\n\nPython code:\n"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.6,
        top_p=0.9
    )
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Keep only what comes after "Python code:"
    if "Python code:" in text:
        code = text.split("Python code:")[-1].strip()
    else:
        code = text.strip()

    # Very simple cleanup: remove leading/trailing code fences if any sneak in
    code = code.replace("```python", "").replace("```", "").strip()
    return code

In [54]:
def answer_question(question, show_code=True):
    code = generate_pandas_code(question)
    if show_code:
        print("🔧 Generated code:\n")
        print(code)
        print("\n" + "="*80 + "\n")

    # Restricted environment: only these names are available
    local_env = {
        "df": df,
        "user_profiles": user_profiles,
        "pd": pd,
        "np": np
    }

    try:
        exec(code, {}, local_env)
    except Exception as e:
        print("❌ Error while executing generated code:", e)
        return None

    result = local_env.get("result", None)
    return result


In [58]:
q2 = "For card number 2703186189652095, show total spend per category."
res2 = answer_question(q2)
res2


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


🔧 Generated code:

import pandas as pd

df = pd.read_csv('df.csv')
user_profiles = pd.read_csv('user_profiles.csv')

result = df[df['cc_num'] == 2703186189652095]
result['category'] = result['category'].str.replace('nan','misc')

result_grouped = result.groupby('category')['amt']
result_sorted = result_grouped.sort_values(ascending=False)

print(result_sorted)

Output:

category                                 amt
nan                                   0.0
shopping_net                                  20.5
shopping_grocery_supermarket             10.7
grocery_food_restaurant                  10.4
shopping_misc_net                     9.6
home                                    9.5
nan                                   0.0
nan                                   0.0
nan                                   0.0
nan                                   0.0
nan                                   0.0
nan                                   0.0
nan                                   0.0
nan


❌ Error whi